In [4]:
import numpy as np

class Tensor():
    def __init__(self, data, autograd=False, creators=None, creation_op=None, id=None):
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
        
        self.autograd = autograd
        if (id is None):
            id = np.random.randint(0, 1000000)
        self.id = id
        
        self.children = {}
        if (creators is not None):
            for c in creators:
                if (self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
    
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if (cnt != 0):
                return False
        return True
    
    def __add__(self, other):
        if (self.autograd and other.autograd):
            return Tensor(self.data + other.data, autograd=True, creators=[self,other], creation_op="add")
        return Tensor(self.data + other.data)
    
    def __neg__(self):
        if (self.autograd):
            return Tensor(self.data * (-1), autograd=True, creators=[self], creation_op="neg")
        return Tensor(self.data * (-1))
    
    def __sub__(self, other):
        if (self.autograd and other.autograd):
            return Tensor(self.data - other.data, autograd=True, creators=[self,other], creation_op="sub")
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        if (self.autograd and other.autograd):
            return Tensor(self.data * other.data, autograd=True, creators=[self,other], creation_op="mul")
        return Tensor(self.data * other.data)
    
    def sum(self, dim):
        if (self.autograd):
            return Tensor(self.data.sum(dim), autograd=True, creators=[self], creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim, copies):
        trans_cmd = list(range(0, len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_shape = list(self.data.shape) + [copies]
        new_data = self.data.repeat(copies).reshape(new_shape)
        new_data = new_data.transpose(trans_cmd)
        if (self.autograd):
            return Tensor(new_data, autograd=True, creators=[self], creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if (self.autograd):
            return Tensor(self.data.transpose(), autograd=True, creators=[self], creation_op="transpose")
        return Tensor(self.data.transpose())
    
    def mm(self, other):
        if (self.autograd and other.autograd):
            return Tensor(self.data.dot(other.data), autograd=True, creators=[self,other], creation_op="mm")
        return Tensor(self.data.dot(other.data))
    
    def sigmoid(self):
        if (self.autograd):
            return Tensor(1.0/(1+np.exp(-self.data)), autograd=True, creators=[self], creation_op="sigmoid")
        return Tensor(1.0/(1+np.exp(-self.data)))
    
    def tanh(self):
        if (self.autograd):
            return Tensor(np.tanh(self.data), autograd=True, creators=[self], creation_op="tanh")
        return Tensor(np.tanh(self.data))
    
    def relu(self):
        if (self.autograd):
            return Tensor((self.data >= 0)*self.data, autograd=True, creators=[self], creation_op="relu")
        return Tensor((self.data >= 0)*self.data)
    
    def softmax(self):
        temp = np.exp(self.data)
        softmax_ouput = temp / np.sum(temp, axis = len(self.data.shape)-1, keepdims=True)
        return softmax_ouput
        
    def cross_entropy(self, target_indices):
        temp = np.exp(self.data)
        softmax_ouput = temp / np.sum(temp, axis = len(self.data.shape)-1, keepdims=True)
        t = target_indices.data.flatten()
        p = softmax_ouput.reshape(len(t),-1)
        target_dist = np.eye(p.shape[1])[t]
        loss = -(np.log(p)*target_dist).sum(1).mean()
        
        cnt = 0
        for i in range(len(p)):
            cnt += int(np.argmax(p[i:i+1]) == t[i])
            
        if (self.autograd):
            out = Tensor(loss, autograd=True, creators=[self], creation_op="cross_entropy")
            out.softmax_ouput = softmax_ouput
            out.target_dist = target_dist
            out.cnt = cnt
            return out
        return Tensor(loss)    
    
    def index_select(self, indices):
        if (self.autograd):
            new = Tensor(self.data[indices.data.flatten()], autograd=True, creators=[self], creation_op="index_select")
            new.index_select_indices = indices
            return new
        return Tensor(self.data[indices.data.flatten()])
    
    def backward(self, grad=None, grad_origin=None):
        if (self.autograd):
            if (grad_origin is not None):
                if (self.children[grad_origin.id] ==0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1
            if (self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
                
            if (self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
                
                if (self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                
                if (self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__(), self)
                
                if (self.creation_op == "sub"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad.__neg__(), self)
                
                if (self.creation_op == "mul"):
                    new = Tensor(self.grad.data * self.creators[1].data)
                    self.creators[0].backward(new, self)
                    new = Tensor(self.grad.data * self.creators[0].data)
                    self.creators[1].backward(new, self)
                
                if (self.creation_op == "mm"):
                    layer = self.creators[0].data
                    weights = self.creators[1].data
                    new = Tensor(self.grad.data.dot(weights.transpose()))
                    self.creators[0].backward(new, self)
                    new = Tensor(layer.transpose().dot(self.grad.data))
                    self.creators[1].backward(new, self)
                
                if (self.creation_op == "transpose"):
                    self.creators[0].backward(self.transpose(), self)
                
                if ("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    copies = self.creators[0].data.shape[dim]
                    self.creators[0].backward(self.grad.expand(dim, copies), self)
                
                if ("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim), self)
                
                if (self.creation_op == "sigmoid"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (self*(ones-self)))
                    
                if (self.creation_op == "tanh"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (ones-self*self))
                    
                if (self.creation_op == "relu"):
                    self.creators[0].backward(self.grad * Tensor(1 * (self.data>=0)))
                    
                if (self.creation_op == "cross_entropy"):
                    dx = self.softmax_ouput - self.target_dist
                    self.creators[0].backward(Tensor(dx))
                    
                if (self.creation_op == "index_select"):
                    new_grad = np.zeros_like(self.creators[0].data)
                    indices_ = self.index_select_indices.data.flatten()
                    grad_ = grad.data.reshape(len(indices_),-1)
                    for i in range(len(indices_)):
                        new_grad[indices_[i]] += grad_[i]
                    self.creators[0].backward(Tensor(new_grad),self)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())
                                        

    
class SGD():
    def __init__(self, parameters, lr=0.1):
        self.parameters = parameters
        self.lr = lr
    
    def step(self, zeros=True):
        for p in self.parameters:
            p.data -= p.grad.data * self.lr # 为什么这里会有.data呀？
            if(zeros):
                p.grad.data *= 0

                
class Layer():
    def __init__(self):
        self.parameters = list()
    def get_parameters(self):
        return self.parameters

    
class Linear(Layer):
    def __init__(self, n_inputs, n_outputs, bias=True):
        super().__init__()
        self.use_bias = bias
        w = np.random.randn(n_inputs,n_outputs)*np.sqrt(2.0/n_inputs)
        self.weights = Tensor(w, autograd=True)
        self.parameters.append(self.weights)
        
        if (self.use_bias):
            self.bias = Tensor(np.zeros(n_outputs), autograd=True)
            self.parameters.append(self.bias)
    
    def forward(self, input):
        if (self.use_bias):
            return input.mm(self.weights) + self.bias.expand(0, len(input.data))
        else:
            return input.mm(self.weights)

        
class Sequential(Layer):
    def __init__(self, layers=list()):
        super().__init__()
        self.layers = layers
    
    def add(self, layers):
        self.layers.append(layer)
        
    def forward(self, input):
        for layer in self.layers:
            input = layer.forward(input)
        return input
    
    def get_parameters(self):
        params = list()
        for l in self.layers:
            params += l.get_parameters()
        return params


class MSELoss(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, pred, target):
        return ((pred-target)*(pred-target)).sum(0)
                                              
class Tanh(Layer):
    def __init__(self):
        super().__init__()
    def forward(self, input):
        return input.tanh()

class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
    def forward(self, input):
        return input.sigmoid()
    
class Relu(Layer):
    def __init__(self):
        super().__init__()
    def forward(self, input):
        return input.relu()
    
class CrossEntropyLoss(Layer):
    def __init__(self):
        super().__init__()
    def forward(self, input, target):
        return input.cross_entropy(target)
    
class Embedding(Layer):
    def __init__(self, vocab_size, dim):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.dim = dim
        
        self.weight = Tensor((np.random.rand(vocab_size, dim) - 0.5)/dim, autograd=True)
        self.parameters.append(self.weight)
        
    def forward(self, input):
        return self.weight.index_select(input)

class RNNCell(Layer):
    def __init__(self, n_inputs, n_hidden, n_output, activation="tanh"):
        super().__init__()
        
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        if (activation == "sigmoid"):
            self.activation = Sigmoid()
        elif (activation == "tanh"):
            self.activation = Tanh()
        elif (activation == "relu"):
            self.activation = Relu()
        else:
            raise Exception("Non-Linearity not found")
        
        self.w_ih = Linear(n_inputs, n_hidden, bias=False)
        self.w_hh = Linear(n_hidden, n_hidden)
        self.w_ho = Linear(n_hidden, n_output)
        
        self.parameters += self.w_ih.get_parameters()
        self.parameters += self.w_hh.get_parameters()
        self.parameters += self.w_ho.get_parameters()
        
    def forward(self, input, hidden, out=True):
        combined = self.w_ih.forward(input) + self.w_hh.forward(hidden)
        new_hidden = self.activation.forward(combined)
        
        output = self.w_ho.forward(new_hidden)
        return output, new_hidden
    
    def init_hidden(self, batch_size=1):
        return Tensor(np.zeros((batch_size, self.n_hidden)), autograd=True)

class LSTMCell(Layer):
    def __init__(self, n_inputs, n_hidden, n_output):
        super().__init__()
        
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        self.xf = Linear(n_inputs, n_hidden, bias=False)
        self.xi = Linear(n_inputs, n_hidden, bias=False)
        self.xo = Linear(n_inputs, n_hidden, bias=False)
        self.xu = Linear(n_inputs, n_hidden, bias=False)
        
        self.hf = Linear(n_hidden, n_hidden)
        self.hi = Linear(n_hidden, n_hidden)
        self.ho = Linear(n_hidden, n_hidden)
        self.hu = Linear(n_hidden, n_hidden)
        
        self.w_ho = Linear(n_hidden, n_output)
        
        self.parameters += self.xf.get_parameters()
        self.parameters += self.xi.get_parameters()
        self.parameters += self.xo.get_parameters()
        self.parameters += self.xu.get_parameters()
        self.parameters += self.hf.get_parameters()
        self.parameters += self.hi.get_parameters()
        self.parameters += self.ho.get_parameters()
        self.parameters += self.hu.get_parameters()
        self.parameters += self.w_ho.get_parameters()
        
    def forward(self, input, hidden):
        prev_hidden = hidden[0]
        prev_cell = hidden[1]
        
        f = (self.xf.forward(input) + self.hf.forward(prev_hidden)).sigmoid()
        i = (self.xi.forward(input) + self.hi.forward(prev_hidden)).sigmoid()
        o = (self.xo.forward(input) + self.ho.forward(prev_hidden)).sigmoid()
        g = (self.xu.forward(input) + self.hu.forward(prev_hidden)).tanh()
        c = (f * prev_cell) + (i * g)
        h = o * c.tanh()
        
        output = self.w_ho.forward(h)
        return output, (h, c)
    
    def init_hidden(self, batch_size=1):
        h = Tensor(np.zeros((batch_size, self.n_hidden)), autograd=True)
        c = Tensor(np.zeros((batch_size, self.n_hidden)), autograd=True)
        h.data[:,0] += 1
        c.data[:,0] += 1
        return (h, c)

class GRUCell(Layer):
    def __init__(self, n_inputs, n_hidden, n_output):
        super().__init__()
        
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        self.xz = Linear(n_inputs, n_hidden, bias=False)
        self.xr = Linear(n_inputs, n_hidden, bias=False)
        self.xs = Linear(n_inputs, n_hidden, bias=False)
        
        self.hz = Linear(n_hidden, n_hidden)
        self.hr = Linear(n_hidden, n_hidden)
        self.hs = Linear(n_hidden, n_hidden)
        
        self.w_ho = Linear(n_hidden, n_output)
        
        self.parameters += self.xz.get_parameters()
        self.parameters += self.xr.get_parameters()
        self.parameters += self.xs.get_parameters()
        self.parameters += self.hz.get_parameters()
        self.parameters += self.hr.get_parameters()
        self.parameters += self.hs.get_parameters()
        
        self.parameters += self.w_ho.get_parameters()
        
    def forward(self, input, hidden):
        
        z = (self.xz.forward(input) + self.hz.forward(hidden)).sigmoid()
        r = (self.xr.forward(input) + self.hr.forward(hidden)).sigmoid()
        s = (self.xs.forward(input) + self.hs.forward(hidden*r)).tanh()
        
        ones = Tensor(np.zeros_like(z.data), autograd=True)
        h = (ones - z) * hidden + z * s
        
        output = self.w_ho.forward(h)
        return output, h
    
    def init_hidden(self, batch_size=1):
        h = Tensor(np.zeros((batch_size, self.n_hidden)), autograd=True)
        return h

In [5]:
np.random.seed(1)
with open("./DataSets/shakespeare.txt", 'r') as file:
    data = file.read()
    
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('Data has %d characters, and %d unique ones.' % (data_size, vocab_size))
char_to_index = {ch:i for i, ch in enumerate(chars)}
index_to_char = {i:ch for i, ch in enumerate(chars)}
indices = np.array(list(map(lambda x:char_to_index[x], data)))

batch_size, bptt = 32, 25
num_batches = int((indices.shape[0]/batch_size))

trimmed_indices = indices[:num_batches*batch_size]
batch_indices = trimmed_indices.reshape(batch_size, num_batches)
batch_indices = batch_indices.transpose()
input_batched_indices = batch_indices[0:-1]
target_batched_indices = batch_indices[1:]

n_bptt = int((num_batches-1)/bptt)
input_batches = input_batched_indices[:n_bptt*bptt]
input_batches = input_batches.reshape(n_bptt, bptt, batch_size)
target_batches = target_batched_indices[:n_bptt*bptt]
target_batches = target_batches.reshape(n_bptt, bptt,batch_size)

Data has 1115394 characters, and 65 unique ones.


In [6]:
embed = Embedding(vocab_size=len(chars), dim=20)
model = LSTMCell(n_inputs=20, n_hidden=256, n_output=len(chars))
model.w_ho.weights.data *= 0

criterion = CrossEntropyLoss()
params = model.get_parameters() + embed.get_parameters()
optim = SGD(parameters=params, lr=0.01)

epochs = 200
for e in range(epochs):
    total_loss = 0
    hidden = model.init_hidden(batch_size=batch_size)
    for batch_i in range(len(input_batches)):
        hidden = (Tensor(hidden[0].data, autograd=True), Tensor(hidden[1].data, autograd=True))
        losses = list()
        for t in range(bptt):
            input = Tensor(input_batches[batch_i][t], autograd=True)
            rnn_input = embed.forward(input=input)
            output, hidden = model.forward(input=rnn_input, hidden=hidden)
            
            target = Tensor(target_batches[batch_i][t], autograd=True)
            batch_loss = criterion.forward(output, target)
            
            if (t == 0):
                losses.append(batch_loss)
            else:
                losses.append(batch_loss + losses[-1])
        loss = losses[-1]
        
        loss.backward()
        optim.step()
        total_loss += loss.data/bptt
        
    optim.lr *= 0.99
    
    if (e%30 == 0 or e == (epochs-1)):
        print("Epoch: %2d - Loss:%0.3f"%(e, total_loss/(batch_i+1)))

Epoch:  0 - Loss:1.784
Epoch: 30 - Loss:1.251
Epoch: 60 - Loss:1.226
Epoch: 90 - Loss:1.189
Epoch: 120 - Loss:1.160
Epoch: 150 - Loss:1.114
Epoch: 180 - Loss:1.083
Epoch: 199 - Loss:1.075


In [ ]:
def sample(init_chars, n):
    hidden = model.init_hidden(batch_size=1)
    s = []
    for t in range(len(init_chars)+n):
        if t < (len(init_chars)):
            ix = char_to_index[init_chars[t]]
            input = Tensor(np.array(ix))
        else:
            output.data *= 5
            prob = output.softmax().ravel()
            # prob = np.log(prob)
            prob = prob/np.sum(prob + 1e-10)
            ix = np.random.choice(range(len(chars)), p=prob)
            input = Tensor(np.array([ix]))
        
        s.append(ix)
        rnn_input = embed.forward(input)
        ouput, hidden = model.forward(input=rnn_input, hidden=hidden)
        
    return s

sample_ix = sample(init_chars=data[0:14],n=50)
s = ''.join(index_to_char[ix] for ix in sample_ix)
print(s)